# DB2-Salesforce connector: Tool usage clusters mapping to users

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolClusterAssociation__c'

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
wang159_myrmekes_db = application.new_db_engine('rfm_data')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

salesforce = application.new_salesforce_engine()
db_s = salesforce


nanoHUB - Serving Students, Researchers & Instructors
Obtained Salesforce access token ...... True


In [2]:
    
import pandas as pd
import time
import datetime

## Obtain tool information from DB2 

In [3]:

class_info_df = pd.read_sql_query('select * from cluster_class_info', wang159_myrmekes_db)
students_info_df = pd.read_sql_query('select * from cluster_students_info', wang159_myrmekes_db)



In [4]:
students_info_df.head(3)

,index,user,class_id,parti_rate
0,0,salas,0,1.0
1,1,trohrbach,0,1.0
2,2,alqadia,0,1.0


## Obtain Salesforce IDs

In [5]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_username_df = db_s.query_data('SELECT Id, nanoHUB_username__c FROM Contact where nanoHUB_username__c != NULL')

[Success] Bulk job creation successful. Job ID = 7505w00000cyX22AAE
{"id":"7505w00000cyX22AAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:31:41.000+0000","systemModstamp":"2022-01-31T03:31:41.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000cyX22AAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:31:41.000+0000","systemModstamp":"2022-01-31T03:31:42.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":19815,"retries":0,"totalProcessingTime":991}
{"id":"7505w00000cyX22AAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03

In [6]:
# get Salesforce ID for citations
sf_clusterID_df = db_s.query_data('SELECT Id, ID__c FROM tool_usage_cluster__c')

sf_clusterID_df['ID__c'] = sf_clusterID_df['ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000cyX1tAAE
{"id":"7505w00000cyX1tAAE","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:32:07.000+0000","systemModstamp":"2022-01-31T03:32:07.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000cyX1tAAE","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2022-01-31T03:32:07.000+0000","systemModstamp":"2022-01-31T03:32:08.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":4576,"retries":0,"totalProcessingTime":413}
[Success] Bulk job completed successfully.


## Match data with Salesforce format

In [7]:
# Display
display(students_info_df.head(2))
display(sf_username_df.head(2))
display(sf_clusterID_df.head(2))

,index,user,class_id,parti_rate
0,0,salas,0,1.0
1,1,trohrbach,0,1.0


,Id,nanoHUB_username__c
0,0035w000031Vsp1AAC,hubrepo
1,0035w000031Vsp2AAC,support


,ID__c,Id
0,4,a0w5w000009Q7CuAAK
1,5,a0w5w000009Q7CvAAK


In [8]:
# merge dataframes
cc_tolink_df = pd.merge(students_info_df, sf_username_df, how='inner', left_on='user', right_on='nanoHUB_username__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

cc_tolink_df = pd.merge(cc_tolink_df, sf_clusterID_df, how='inner', left_on='class_id', right_on='ID__c')\
                           .rename(columns={'Id':'SF_ID_cluster'})

# display
cc_tolink_df.head(2)

,index,user,class_id,parti_rate,SF_ID_contact,nanoHUB_username__c,ID__c,SF_ID_cluster
0,0,salas,0,1.0,0035w000034Jm4mAAC,salas,0,a0w5w00000A8U1UAAV
1,1,trohrbach,0,1.0,0035w000034Jm55AAC,trohrbach,0,a0w5w00000A8U1UAAV


In [9]:
# create a new column for object name
cc_tolink_df['Name'] = cc_tolink_df.apply(lambda x: '%s_%d'%(x.nanoHUB_username__c, x.ID__c), axis=1)

In [10]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = cc_tolink_df['Name']
df_sf['Contact__c'] = cc_tolink_df['SF_ID_contact']
df_sf['Tool_Usage_Cluster__c'] = cc_tolink_df['SF_ID_cluster']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,salas_0,trohrbach_0
Contact__c,0035w000034Jm4mAAC,0035w000034Jm55AAC
Tool_Usage_Cluster__c,a0w5w00000A8U1UAAV,a0w5w00000A8U1UAAV


In [11]:
df_sf.shape

(103626, 3)

## To Salesforce Sales Cloud CRM

In [12]:
db_s = salesforce

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

In [13]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000cyX27AAE
hello
[Success] CSV upload successful. Job ID = 7505w00000cyX27AAE
[Success] Closing job successful. Job ID = 7505w00000cyX27AAE


In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2022-01-31T03:32:22.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000cyX27AAE',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'ContactToolClusterAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'InProgress',
 'systemModstamp': '2022-01-31T03:32:25.000+0000',
 'totalProcessingTime': 0}
